Well, in [002_calc_some_stats.ipynb](https://github.com/dosquisd/NMDB-FD-PredictorWithGraphs/blob/5952726527e9855ce448bb8a9d8ad7a7f33317ba/notebooks/002_calc_some_stats.ipynb) there was a lot of code, mixing creating dataset and plotting them, so I created this notebook to have a dedicated notebook, only to create dataset

In [12]:
%%capture
%load_ext autoreload
%autoreload 2

In [13]:
import itertools
import logging
from typing import Any, Callable, Dict, List, Optional, Tuple

import networkx as nx
import nolds
import numpy as np
import pandas as pd
from scipy.stats import entropy

from utils import (
    MIN_VALUE_THRESHOLD,
    ROOTDIR,
    AdjacencyMethod,
    DistanceTransformation,
    EventData,
    GraphEvent,
    Normalizer,
    encode_variables_to_filename,
    get_events,
    graph_fractal_dimension,
    logger,
    read_dataset,
)

logger.setLevel(logging.DEBUG)

## Create dataset

In [14]:
def calc_graph_metrics(
    events: Dict[str, EventData],
    *,
    threshold: float = 0.0,
    valid_methods: Optional[List[AdjacencyMethod]] = None,
) -> Tuple[pd.DataFrame, List[Dict[str, Any]]]:
    def calculate_avg_node_metric(
        metrics: Dict[str, float],
        fn: Optional[Callable[[np.ndarray], float]] = None,
        *,
        value_per_node: Optional[Dict[str, float]] = None,
    ) -> float:
        if fn is None:
            fn = lambda nums: float(nums.mean())  # noqa: E731

        if value_per_node is None:
            value_per_node = {}

        values = np.array(
            list(
                map(
                    lambda item: item[1] * value_per_node.get(item[0], 1.0),
                    metrics.items(),
                )
            )
        )
        return fn(values)

    if valid_methods is None:
        valid_methods = [AdjacencyMethod.MANHATTAN, AdjacencyMethod.MINKOWSKI]

    # The same amount of iterations, but at least there's only one loop instead of 4 nested loops
    dataset = []
    combinations = itertools.product(
        events.keys(), DistanceTransformation, Normalizer, valid_methods
    )
    for event_date, transform_method, normalization, adj_method in combinations:
        data = events[event_date]
        raw_df = data["raw"].reset_index(drop=True)
        logger.info(
            f"Processing event: {event_date} with transformation: {transform_method.value}, "
            f"normalization: {normalization.value}, adjacency method: {adj_method.value}, "
            f"raw shape: {raw_df.shape}"
        )

        if transform_method == DistanceTransformation.LOG:
            raw_df[raw_df.abs() < MIN_VALUE_THRESHOLD] = MIN_VALUE_THRESHOLD

        # Transform raw_df data before any normalization
        transformed_data = transform_method.transform(raw_df.to_numpy())
        transformed_df = pd.DataFrame(transformed_data, columns=raw_df.columns)
        transformed_df[transformed_df.abs() < MIN_VALUE_THRESHOLD] = MIN_VALUE_THRESHOLD

        # Normalize data
        columns = transformed_df.columns
        normalized_data = normalization.normalize(transformed_df.to_numpy())
        normalized_df = pd.DataFrame(normalized_data, columns=columns)
        nan_sum = normalized_df.isna().sum()
        nan_columns = nan_sum[nan_sum > 0].index.tolist()
        if nan_columns:
            logger.warning(f"NAN COLUMNS DROPPED: {nan_columns}")
            normalized_df.drop(columns=nan_columns, inplace=True)

        # normalized_df[normalized_df.abs() < MIN_VALUE_THRESHOLD] = MIN_VALUE_THRESHOLD

        # Create graph event with the normalized data and metadata
        graph_event = GraphEvent(
            data=normalized_df,
            metadata={
                # Event metadata
                "drop": data.get("drop", 0.0),
                "intensity": data.get("intensity", "Unknown"),
                "dst": data.get("dst", 0.0),
                # Station metadata
                "cutoff_rigidity": data["cutoff_rigidity"],
                "altitude": data["altitude"],
            },
        )

        try:
            graph = graph_event.get_graph_networkx(adj_method, threshold=threshold)
        except Exception as e:
            logger.error(
                f"Error processing event {event_date} with method {adj_method.value}: {e}"
            )

            print("\nraw_df snapshot:")
            nan_sum = raw_df.isna().sum()
            display(nan_sum[nan_sum > 0])

            print("transformed_df snapshot:")
            nan_sum = transformed_df.isna().sum()
            display(transformed_df)
            display(nan_sum[nan_sum > 0])

            print("normalized_df snapshot:")
            nan_sum = normalized_df.isna().sum()
            display(normalized_df)
            display(nan_sum[nan_sum > 0])

            raise e

        # Calculate MST and store it
        graph = nx.minimum_spanning_tree(graph)
        events[event_date]["graphs"][adj_method] = graph

        # Weight distribution in order to calculate other metrics
        weights = np.array(
            list(map(lambda item: item[2]["weight"], graph.edges(data=True)))
        )

        # Global graph metrics
        dataset.append(
            {
                "event_date": event_date,
                # Event metadata
                "drop": data.get("drop", 0.0),
                "intensity": data.get("intensity", "Unknown"),
                "dst": data.get("dst", 0.0),
                # Metric metadata
                "transformation": transform_method.value,
                "normalization": normalization.value,
                "adjacency_method": adj_method.value,
                "graph": graph,
                # Graph global metrics
                "global_efficiency": nx.global_efficiency(graph),
                "estrada_index": nx.estrada_index(graph),
                "entropy": entropy(weights),
                "fractal": graph_fractal_dimension(graph, seed=37)[0],
                "hurst_rs": nolds.hurst_rs(weights, fit="poly"),
                "modularity": nx.algorithms.community.modularity(
                    graph,
                    list(nx.algorithms.community.greedy_modularity_communities(graph)),
                ),
                "assortativity": nx.degree_assortativity_coefficient(graph),
                # Average node metrics
                "avg_katz": calculate_avg_node_metric(nx.katz_centrality(graph)),
                "avg_closeness": calculate_avg_node_metric(
                    nx.closeness_centrality(graph)
                ),
                "avg_betweenness": calculate_avg_node_metric(
                    nx.betweenness_centrality(graph)
                ),
                "avg_laplacian": calculate_avg_node_metric(
                    nx.laplacian_centrality(graph)
                ),
            }
        )

    dataset_df = pd.DataFrame(dataset)
    return dataset_df, dataset

### Save datasets

In [ ]:
valid_adj_methods = [
    AdjacencyMethod.MANHATTAN,
    AdjacencyMethod.MINKOWSKI,
]

# Options for processing different files and configurations
filename_options = ["all.txt", "all.original.txt", "all.imp.txt"]
imput_data_options = [False, True]
use_threshold_options = [False, True]

options = itertools.product(filename_options, imput_data_options, use_threshold_options)
for filename, imput_data, use_threshold in options:
    if use_threshold != imput_data:
        logger.info(
            f"Skipping file: {filename} with imput_data={imput_data} and "
            f"use_threshold={use_threshold} (invalid combination)"
        )
        continue

    logger.info(f"Processing file: {filename} with imput_data={imput_data}")
    events = get_events(
        filename=filename, imput_data=imput_data, use_threshold=use_threshold
    )
    dataset_df, dataset = calc_graph_metrics(events, valid_methods=valid_adj_methods)

    logger.info(f"Saving dataset for file: {filename} with imput_data={imput_data}")
    dataset_df.drop(columns=["graph"]).to_csv(
        ROOTDIR
        / "data"
        / encode_variables_to_filename(
            event_filename=filename, imput_data=imput_data, use_threshold=use_threshold
        ),
        index=False,
    )

## Read datasets to ensure are created properly

In [16]:
read_dataset(ROOTDIR / "data" / "dataset_all_imput-False_threshold-False.csv")

,event_date,drop,intensity,dst,transformation,normalization,adjacency_method,global_efficiency,estrada_index,entropy,fractal,hurst_rs,modularity,assortativity,avg_katz,avg_closeness,avg_betweenness,avg_laplacian,graph
0,2024-05-10,15.32,G5,-412.0,none,min_max,manhattan,0.282992,73.135234,3.343178,1.278966,0.570866,0.668313,-0.314667,0.180939,0.186933,0.165764,0.081755,"(MXCO, NANM, CALM, ROME, AATB, BKSN, JUNG, JUN..."
1,2024-05-10,15.32,G5,-412.0,none,min_max,minkowski,0.284762,72.870807,3.343198,1.322432,0.529297,0.669813,-0.295971,0.180983,0.192791,0.158128,0.081376,"(MXCO, NANM, CALM, ROME, AATB, BKSN, JUNG, JUN..."
2,2024-05-10,15.32,G5,-412.0,none,z_score,manhattan,0.291746,73.483465,3.336238,1.216267,0.204672,0.676643,-0.393393,0.180861,0.203899,0.145402,0.081349,"(MXCO, NANM, CALM, ROME, AATB, BKSN, JUNG, JUN..."
3,2024-05-10,15.32,G5,-412.0,none,z_score,minkowski,0.270104,71.694679,3.335221,1.263404,0.457441,0.671031,-0.418754,0.181306,0.172011,0.178407,0.080409,"(MXCO, NANM, CALM, ROME, AATB, BKSN, JUNG, JUN..."
4,2024-05-10,15.32,G5,-412.0,none,robust,manhattan,0.293525,72.553556,3.320189,1.382974,0.301323,0.672425,-0.293582,0.181019,0.210933,0.139655,0.080683,"(MXCO, NANM, CALM, ROME, AATB, BKSN, JUNG, JUN..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015,2005-09-11,12.25,G3,-139.0,exponential,robust,minkowski,0.370329,33.360243,2.587530,1.406931,0.470459,0.518883,-0.340426,0.257540,0.268541,0.223443,0.155391,"(ESOI, MXCO, ROME, AATB, LMKS, MOSC, NEWK, CAL..."
1016,2005-09-11,12.25,G3,-139.0,exponential,decimal_scaling,manhattan,0.376799,33.796712,2.626843,1.280921,0.600347,0.529771,-0.477778,0.257264,0.271259,0.220513,0.158636,"(ESOI, MXCO, ROME, AATB, LMKS, MOSC, NEWK, CAL..."
1017,2005-09-11,12.25,G3,-139.0,exponential,decimal_scaling,minkowski,0.385665,34.457961,2.618426,1.256231,0.519619,0.528008,-0.607843,0.256903,0.279565,0.211722,0.160281,"(ESOI, MXCO, ROME, AATB, LMKS, MOSC, NEWK, CAL..."
1018,2005-09-11,12.25,G3,-139.0,exponential,none,manhattan,0.376799,33.796712,2.626843,1.280921,0.600347,0.529771,-0.477778,0.257264,0.271259,0.220513,0.158636,"(ESOI, MXCO, ROME, AATB, LMKS, MOSC, NEWK, CAL..."
